In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def visualize_normal_mode(geometry, eigenvector, scale=10):
    vec = eigenvector.reshape(-1, 3)
    trace2 = go.Cone(
        x=geometry[:, 0],
        y=geometry[:, 1],
        z=geometry[:, 2],
        u=vec[:, 0] * scale,
        v=vec[:, 1] * scale,
        w=vec[:, 2] * scale,
        sizemode="absolute", # "absolute"
        sizeref=2,
        anchor="tail"
    )
    return [trace2]

def visualize_normal_modes(geometry, eigenvectors, scale=10, cols=3):
#     eigenvectors = eigenvectors.round(2)
    N3, N3 = eigenvectors.shape
    rows = int(N3/cols)
    if N3 % cols > 0:
        rows += 1
    specs = [[{'is_3d': True} for i in range(cols)]
             for j in range(rows)]
    fig = make_subplots(rows=rows, cols=cols, specs=specs)
    for row in range(rows):
        for col in range(cols):
            i = row * cols + col
            if i >= N3:
                continue
            traces = visualize_normal_mode(geometry, eigenvectors[:, i], scale)
            fig.add_trace(traces[0], row=row + 1, col=col + 1)
    fig.update_layout(scene_aspectmode='data')
    return fig

## Examine hinged triangles

In [ ]:
R = np.array([
    [-1., 0, 1],
    [0, 1, 0],
    [0, -1, 0],
    [1, 0, 1],
])

In [ ]:
springs = [(0, 1), (1, 2), (2, 0), (3, 2), (3, 1)]

In [ ]:
mySum = np.sum(R, 0)
for i in range(len(R)):
    R[i] = R[i] - mySum/len(R)

In [ ]:
H = np.zeros((len(R)*3, len(R)*3))
for i, (start, stop) in enumerate(springs):
    proj = R[stop]-R[start]
    proj /= np.linalg.norm(proj)
    proj = np.abs(proj)
    
    H[start*3:start*3+3, start*3:start*3+3] += np.eye(3)*proj
    H[stop*3:stop*3+3, stop*3:stop*3+3] += np.eye(3)*proj
    
    H[start*3:start*3+3, stop*3:stop*3+3] += -np.eye(3)*proj
    H[stop*3:stop*3+3, start*3:start*3+3] += -np.eye(3)*proj
    

In [ ]:
l, v = np.linalg.eigh(H)

v = v.T

In [ ]:
l.round(3)

In [ ]:
visualize_normal_modes(R, v.T, cols=4)

In [ ]:
def inertiaAxes(x,y,z):
    I = np.empty((3,3))
    
    I[0,0] = np.sum(y**2+z**2)
    I[1,1] = np.sum(x**2+z**2)
    I[2,2] = np.sum(x**2+y**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    I[0,2] = I[2,0] = -np.sum(z*x)
    I[1,2] = I[2,1] = -np.sum(y*z)
    
    return I

In [ ]:
I = inertiaAxes(R[:,0], R[:,1], R[:,2])
I_prime, X = np.linalg.eigh(I)
D = np.empty((6,len(R)*3))

In [ ]:
for i in range(len(R)):
    for j in range(3):
        D[3,i*3+j] = np.dot(R[i],X[1])*X[j,2]-np.dot(R[i],X[2])*X[j,1]
        D[4,i*3+j] = np.dot(R[i],X[2])*X[j,0]-np.dot(R[i],X[0])*X[j,2]  
        D[5,i*3+j] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:3,i*3:i*3+3] = np.eye(3)

In [ ]:
for i in range(len(D)):
    D[i] /= np.linalg.norm(D[i])

In [ ]:
for i in range(len(v)):
    remainder = v[i] - np.sum((D @ v[i]).reshape(-1,1) * D, 0)

    if np.linalg.norm(remainder)<.5:
        v[i] = np.zeros(len(v[i]))
    else:
        remainder /= np.linalg.norm(remainder)
        v[i] = remainder
        D = np.append(D, remainder.reshape(1,-1), 0)

In [ ]:
visualize_normal_modes(R, D.T, cols=4)

## This is 0 bc it is a 0 eigenvalue eigenvector!!!

In [ ]:
D[6].round(4)

In [ ]:
(H @ D[6]).round(3)

In [ ]:
np.allclose((D.T @ D), np.eye(len(D)))

In [ ]:
z = plt.imshow(H, vmin=-3, vmax=3, cmap='RdBu')
c = plt.colorbar(z)

In [ ]:
z = plt.imshow(D @ H @ D.T, vmin=-3, vmax=3, cmap='RdBu')
c = plt.colorbar(z)

In [ ]:
np.diag(D @ H @ D.T).round(3)

In [ ]:
(H @ D[5]).round(3)

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D))

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D)[6:,6:])

In [ ]:
l_new.round(3)

In [ ]:
new_D = D[6:]

In [ ]:
np.allclose(new_D @ new_D.T, np.eye(6))

In [ ]:
v_new.shape # [new_eigen, old_eigen]

In [ ]:
new_D.shape # [old_eigen, displacements (xyz)]

In [ ]:
visualize_normal_modes(R, np.einsum('no,od->dn', v_new, new_D), cols=4)

In [ ]:
D[6:].round(2)

In [ ]:
np.allclose(D[6:,6:] @ D[6:,6:].T, np.eye(len(D)-6))

In [ ]:
l_new.round(4)

In [ ]:
v_new[5].shape